<a href="https://colab.research.google.com/github/1hamzaiqbal/MFCLIP_acv/blob/hamza%2Fdiscrim/vit_generator_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) GPU + repo
!nvidia-smi
%cd /content
!git clone -b hamza/discrim https://github.com/1hamzaiqbal/MFCLIP_acv
%cd MFCLIP_acv

In [ ]:
# 1) Deps
!pip install torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy timm



In [ ]:
# 2) Data (example: Oxford Pets into /content/data/oxford_pets)
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from pathlib import Path
root = Path("/content/data/oxford_pets")
root.mkdir(parents=True, exist_ok=True)
_ = OxfordIIITPet(root=str(root), download=True, transform=transforms.ToTensor())
print("Oxford Pets downloaded to", root)



In [ ]:
# fetch official Oxford-IIIT Pet dataset (provides annotations/trainval.txt)
%cd /content
!mkdir -p /content/data/oxford_pets
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -q https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xf images.tar.gz -C /content/data/oxford_pets
!tar -xf annotations.tar.gz -C /content/data/oxford_pets
!ls /content/data/oxford_pets/annotations | head


In [ ]:
# 3) Mount Drive and Setup Checkpoint
from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

# Define paths
src_ckpt = "/content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets/RN50_ArcFace_oxford_pets.pth"
dst_dir = "/content/data/oxford_pets"
dst_ckpt = os.path.join(dst_dir, "RN50_ArcFace.pth")

# Copy checkpoint
os.makedirs(dst_dir, exist_ok=True)
if os.path.exists(src_ckpt):
    shutil.copy(src_ckpt, dst_ckpt)
    print(f"Copied {src_ckpt} to {dst_ckpt}")
else:
    print(f"Warning: Checkpoint not found at {src_ckpt}. Please ensure Drive is mounted and path is correct.")


In [60]:
# 4) Train ViT Generator
!python main.py \
  --flag train_unet \
  --generator vit \
  --dataset oxford_pets \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
  --surrogate RN50 \
  --head ArcFace \
  --num_epoch 1 \
  --bs 64 \
  --lr 0.01 \
  --optimizer SGD \
  --ratio 0.2 \
  --device cuda:0


/content/MFCLIP_acv
2025-11-30 05:07:41.967409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764479261.988662   12400 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764479261.995082   12400 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764479262.011586   12400 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764479262.011619   12400 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764479262.011622   12400 computation_placer.cc:177] co

In [61]:
# 5) Save Artifacts and Plot History
import json
import matplotlib.pyplot as plt
import shutil
import os

# Paths
src_model = "/content/data/oxford_pets/vit_generator.pt"
src_history = "/content/data/oxford_pets/vit_generator_history.json"
dst_dir = "/content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets"

# Copy to Drive
os.makedirs(dst_dir, exist_ok=True)
if os.path.exists(src_model):
    shutil.copy(src_model, os.path.join(dst_dir, "vit_generator.pt"))
    print(f"Saved model to {dst_dir}/vit_generator.pt")
else:
    print("Model file not found!")

if os.path.exists(src_history):
    shutil.copy(src_history, os.path.join(dst_dir, "vit_generator_history.json"))
    print(f"Saved history to {dst_dir}/vit_generator_history.json")

    # Plot
    with open(src_history, 'r') as f:
        history = json.load(f)

    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color=color)
    ax1.plot(history['epoch'], history['loss'], color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Accuracy', color=color)
    ax2.plot(history['epoch'], history['acc'], color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title("Training Progress")
    plt.show()
else:
    print("History file not found!")


Saved model to /content/drive/MyDrive/grad/comp_vision/hanson_loss/oxford_pets/vit_generator.pt
History file not found!


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
